# Assign overlap p values to individual proteins and perform multiple testing correction

In [1]:
import pandas as pd
import numpy as np
import statsmodels.stats.multitest

In [2]:
CHROMOSOMES = ["8"]
ARMS = ["p", "q"]
TRANS_OR_CIS_OPTS = ["cis", "trans"]

## Load files

In [3]:
all_overlaps = pd.DataFrame()
all_pvals = pd.DataFrame()

for CHROMOSOME in CHROMOSOMES:
    for ARM in ARMS:
        for TRANS_OR_CIS in TRANS_OR_CIS_OPTS:
            
            overlaps = pd.\
            read_csv(f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t").\
            assign(group=f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}")
            
            pvals = pd.\
            read_csv(f"overlap_pvals_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t").\
            assign(group=f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}")
            
            all_overlaps = all_overlaps.append(overlaps)
            all_pvals = all_pvals.append(pvals)

## Merge in the _p_ values for each protein

We will exclude proteins that were only different in one cancer type (no overlap).

In [4]:
all_overlaps = all_overlaps[all_overlaps["num_cancers"] > 1].\
merge(
    right=all_pvals,
    left_on=["num_cancers", "group"],
    right_on=["overlap_size", "group"],
    how="inner"
)

## Apply multiple testing correction

In [5]:
reject, adj_pvals, alpha_sidak, alpha_bonf = statsmodels.stats.multitest.multipletests(
    pvals=all_overlaps["pvals"], 
    alpha=0.05, 
    method="fdr_bh"
)

all_overlaps = all_overlaps.\
assign(adj_p=adj_pvals)

all_overlaps = all_overlaps[all_overlaps["adj_p"] <= 0.05].\
sort_values(by=["adj_p", "protein", "group"])

## Split table by protein group

In [6]:
groups = { group:df.reset_index(drop=True) for group, df in all_overlaps.groupby("group")}
pd.options.display.max_rows = None

## Print results

In [7]:
groups["8p_cis"]

,protein,cancers,mean_simp_change,num_cancers,group,overlap_size,pvals,adj_p
0,ATP6V1B2,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415
1,CHMP7,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415
2,ERI1,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415
3,KIF13B,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415
4,PPP2R2A,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415
5,VPS37A,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415
6,XPO7,brca_colon_hnscc_lscc_luad_ovarian,-1,6,8p_cis,6,0.004598,0.028415


In [8]:
groups["8p_trans"]

,protein,cancers,mean_simp_change,num_cancers,group,overlap_size,pvals,adj_p
0,ATP6V1H,colon_lscc_luad,-1,3,8p_trans,3,0.000012,0.000325
1,CNOT8,brca_lscc_luad,1,3,8p_trans,3,0.000012,0.000325
2,ATP6V1E1,colon_luad,-1,2,8p_trans,2,0.008317,0.043572
3,ATP6V1G1,colon_lscc,-1,2,8p_trans,2,0.008317,0.043572
4,HIST1H1D,hnscc_luad,-1,2,8p_trans,2,0.008317,0.043572
5,MTMR6,lscc_luad,-1,2,8p_trans,2,0.008317,0.043572
6,PPIB,lscc_luad,-1,2,8p_trans,2,0.008317,0.043572
7,SMAP2,hnscc_lscc,-1,2,8p_trans,2,0.008317,0.043572
8,TXNDC5,lscc_luad,-1,2,8p_trans,2,0.008317,0.043572


In [9]:
groups["8q_cis"]

,protein,cancers,mean_simp_change,num_cancers,group,overlap_size,pvals,adj_p
0,CPNE3,brca_colon_hnscc_lscc_luad_ovarian,1,6,8q_cis,6,0.001177,0.008868
1,OTUD6B,brca_colon_hnscc_lscc_luad_ovarian,1,6,8q_cis,6,0.001177,0.008868
2,POP1,brca_colon_hnscc_lscc_luad_ovarian,1,6,8q_cis,6,0.001177,0.008868
3,RMDN1,brca_colon_hnscc_lscc_luad_ovarian,1,6,8q_cis,6,0.001177,0.008868
4,YWHAZ,brca_colon_hnscc_lscc_luad_ovarian,1,6,8q_cis,6,0.001177,0.008868


In [10]:
groups["8q_trans"]

,protein,cancers,mean_simp_change,num_cancers,group,overlap_size,pvals,adj_p
0,ARFGEF1,brca_colon_hnscc_lscc_luad,1,5,8q_trans,5,0.000000,0.000000
1,HGH1,brca_colon_hnscc_lscc_luad,1,5,8q_trans,5,0.000000,0.000000
2,STAU2,brca_colon_hnscc_lscc_luad,1,5,8q_trans,5,0.000000,0.000000
3,YTHDF3,brca_colon_hnscc_lscc_luad,1,5,8q_trans,5,0.000000,0.000000
4,GGH,brca_colon_hnscc_luad,1,4,8q_trans,4,0.000011,0.000325
5,MTFR1,brca_hnscc_lscc_luad,1,4,8q_trans,4,0.000011,0.000325
6,PRKDC,brca_colon_lscc_luad,1,4,8q_trans,4,0.000011,0.000325
7,ADNP,colon_lscc_luad,1,3,8q_trans,3,0.000995,0.008868
8,ANKFY1,brca_colon_lscc,-1,3,8q_trans,3,0.000995,0.008868
9,BOP1,brca_colon_luad,1,3,8q_trans,3,0.000995,0.008868


## Save results

In [11]:
all_overlaps.to_csv("sig_diff_all_overlaps.tsv", sep="\t", index=False)